# Lesson 10 


First 15 minutes of the lecture are a showcase of student work and then a review / recap of how stable diffusion works. 


### Recent Papers 
Then he talks about two paper that came out recently (relative to the lecture)
[Progressive Distillation for Fast Sampling of Diffusion Models](https://arxiv.org/abs/2202.00512) [paper walkthrough](https://www.youtube.com/watch?v=ZXuK6IRJlnk) .
This takes use from 60 steps of denoising to just a few using 'distillation' of the model (which is a form of transfer learning).

The teacher model is our fully trained stable diffusion model.  Then a student model (a unnet) is trained to skip over steps prgressively.  

Another paper from same group shows how to use distillation for guided diffusion models:
[On Distillation of Guided Diffusion Models](https://arxiv.org/abs/2210.03142)



 [Imagic: Text-Based Real Image Editing with Diffusion Models](https://arxiv.org/abs/2210.09276)

 The idea here is that you give the model an image and a text prompt.  It uses the text prompt to modify the image. For example, a picture of a dog and the prompt "A dog sitting" will create a picture of the same dog, sitting.

 The method to make this work is:

 1- Train the embedding of the text prompt to be close to the the image.   This finds an embedding that is close the image but also close to the text prompt.

 2 - Fine tune the whole model, using the new embedding, to the image. 
 
 3 - Use the model to generate the new image, interpolating between the original text embedding and the new one.. 


And, there is also this paper [Intruct Pix to Pix](https://arxiv.org/abs/2211.09800) that does the same kind of thing, except instead of providing a label of the final image, you provide *instructions*.   (e.g. "Add fireworks to the sky"). This is a specially trained model, using a dataset of images and instructions that were created from stable diffusion and GPT models.  

## Stable Diffusion walkthrough

(Around 33 minutes in)
The next part of the video continues to walk throuhg `stable_diffusion.ipynb` on [github](https://github.com/fastai/diffusion-nbs)

This digs into the pipeline more, seperately creating the text embeddings and using the models unet to generate the diffusion steps.  I am not goint to repeat it here.

Some notes:

- The Unet needs to know the 'timestep', that is the amount of noise that was added to the image. As Jeremy said last week this was soemthing he brushed over, but the model gets this information during training, so we have to pass it in during inference as well, following the same schedule.  

- Also as discussed last week, the model does inference on both guided and unguided diffusion. [Classifier free guidance](https://arxiv.org/abs/2207.12598) These are interpolated (in the latent space) by the `guidance scale` parameter.   The videos so far have not gone into the details of how this works, but it is in the paper (and there are other videos that do go into it).  Perhaps it is in the extra video [Stable Diffusion Deep Dive](https://www.youtube.com/watch?v=844LY0vYQhc).  The basic idea is that simply denoising with text guidance is not enough,and so the trick that is used is to denoise with guidance and without guidance.  That take the difference and multiply it by some number >1 (`guidance scale`) to exaggerate the difference.   This is added to the unguided result 'extrapolate' the guidance.  This is done at each timestep.


## Back to to foundations. . 
(at 53:30) 

Notebooks for this part are at [Course22p2](https://github.com/fastai/course22p2)